<a href="https://colab.research.google.com/github/laurahallaman1/Pandas-Challenge/blob/master/Pymoli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Import Dependencies 
import pandas as pd 



In [11]:
#Load data CSV 

pymoli_file = "/Users/lauraehallaman/Documents/GitHub/Pandas-Challenge/purchase_data.csv"
pymoli_purchase = pd.read_csv(pymoli_file)


FileNotFoundError: ignored

In [0]:
# Player Count

#Total Number of Players




In [0]:
#Purchasing Analysis (Total)

#Number of Unique Items
#Average Purchase Price
#Total Number of Purchases
#Total Revenue




In [0]:
#Gender Demographics

#Percentage and Count of Male Players
#Percentage and Count of Female Players
#Percentage and Count of Other / Non-Disclosed




In [0]:
#Purchasing Analysis (Gender)
#The below each broken by gender

#Purchase Count
#Average Purchase Price
#Total Purchase Value
#Average Purchase Total per Person by Gender






In [0]:
#Age Demographics

#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

#Purchase Count
#Average Purchase Price
#Total Purchase Value
#Average Purchase Total per Person by Age Group






In [0]:
#Top Spenders

#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

#SN
#Purchase Count
#Average Purchase Price
#Total Purchase Value



In [0]:
#Most Popular Items

#Identify the 5 most popular items by purchase count, then list (in a table):

#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value


In [0]:
#Most Profitable Items

#Identify the 5 most profitable items by total purchase value, then list (in a table):

#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value


In [0]:
#As final considerations:

#You must use the Pandas Library and the Jupyter Notebook.
#You must submit a link to your Jupyter Notebook with the viewable Data Frames.
#You must include a written description of three observable trends based on the data.
#See Example Solution for a reference on expected format.